In [3]:
%load_ext autoreload
%autoreload 2

%load_ext blackcellmagic

In [1]:
import pyphi
from visualize_pyphi import utils
from visualize_pyphi import network_generator
from visualize_pyphi.utils import ces2df
import string
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np
import itertools
import toolz
import pandas as pd

import glob

"""
Check that the config parameters are correct (if not change pyphi_config.yml to match):
REPERTOIRE_DISTANCE = 'maximal-state-first'
IIT_VERSION = 'IIT_4.0_SMALL_PHI'
PARTITION_TYPE = 'TRI'
"""

print(
    pyphi.config.REPERTOIRE_DISTANCE,
    pyphi.config.IIT_VERSION,
    pyphi.config.PARTITION_TYPE,
    pyphi.config.RELATION_PARTITION_TYPE,
    pyphi.config.RELATION_PARTITION_AGGREGATION,
    pyphi.config.SYSTEM_PARTITION_TYPE,
    pyphi.config.PROGRESS_BARS,
    pyphi.config.RELATION_POTENTIAL_PURVIEWS,
    pyphi.config.RELATION_COMPUTATION,
    pyphi.config.RELATION_APPROXIMATION,
    pyphi.config.RELATION_PHI_SCHEME,
    pyphi.config.RELATION_APPROXIMATION_SAMPLE_TIMEOUT
)

IIT_4.0_SMALL_PHI maximal-state-first TRI BI_CUT_ONE SUM TEMPORAL_DIRECTED_BI_CUT_ONE False WHOLE EXACT SAMPLED_DEGREE_ONE OVERLAP_RATIO_TIMES_RELATION_INFORMATIVENESS 1.0


In [5]:
# Define number of nodes, determinism level (k) and self-loop strenght (s) 
n = 3
k = 5
s = .5

"""
Args:
- weight_distributions: "L", "pareto", "nearest_neighbor"
- use self_loop_value var if weight_distribution == "L" or "nearest_neighbor"
- use weight_decay_value var if weight_distribution == "pareto"
"""

grid = network_generator.ToroidalGrid(
    n_nodes=n,
    determinism_value=k,
    weight_distribution="nearest_neighbor",
    self_loop_value=s,
    backgound_conditions=True,
)

subsystem = grid.subsystem

ces = pyphi.compute.ces(subsystem, parallel=True)

In [6]:
ces2df(ces)

,mechanism,state,cause,state,phi,effect,state,phi
1,A,0,A,0,0.343415,A,0,0.416244
2,B,0,B,0,0.427973,AB,00,0.428097
3,C,0,C,0,0.343415,C,0,0.416244
4,AB,00,A,0,0.124558,AB,00,0.244884
5,AC,00,B,0,0.126664,B,0,0.165641
6,BC,00,C,0,0.124558,BC,00,0.244884
7,ABC,000,AC,00,0.073735,ABC,000,0.188518


In [7]:
relations = pyphi.relations.relations(subsystem, ces)

  0%|          | 0/291 [00:00<?, ?it/s]

In [8]:
sia = pyphi.big_phi.sia(
    subsystem,
    ces,
    relations,
)

2022-03-07 15:39:50,271	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8269


Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state


In [13]:
print(f"""
k = {k}
s = {s}
big phi = {sia.phi}
\ndistinctions:\n {ces2df(sia.phi_structure.distinctions)}
""")


k = 5
s = 0.5
big phi = 0.0

distinctions:
   mechanism state cause state       phi effect state       phi
1         A     0     A     0  0.343415      A     0  0.416244
2         B     0     B     0  0.427973     AB    00  0.428097
3         C     0     C     0  0.343415      C     0  0.416244
4       ABC   000    AC    00  0.073735    ABC   000  0.188518

